# EDA

This notebook contains the EDA for the  [NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), specifically for **Yellow** Taxi Trip Records.

The data is stored in parquet format. Let´s take a look.

In [1]:
# Import libraries
import pandas as pd
import os
import numpy as np

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2022.

In [2]:
taxi_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')
taxi_feb = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [3]:
taxi_jan.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
taxi_feb.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


Read the data for January. How many columns are there?

In [5]:
taxi_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

We can see that there are 18 columns.

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

In [6]:
# create duration variable computing the difference between dropoff and pickup times
taxi_jan['duration'] = taxi_jan.tpep_dropoff_datetime - taxi_jan.tpep_pickup_datetime 

# convert it to minutes
taxi_jan['duration'] = taxi_jan['duration'].dt.total_seconds()/60

# obtain std
print(f"The standard deviation of the trips in January is {taxi_jan['duration'].std():.3f} minutes")

The standard deviation of the trips in January is 42.594 minutes


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [33]:
# remove outliers
taxi_jan_filtered = taxi_jan[(taxi_jan['duration'] >= 1.0) & (taxi_jan['duration'] <= 60.0)]

# obtain remaining percentage
remaining_perc = taxi_jan_filtered.shape[0]/taxi_jan.shape[0] 

print(f"{remaining_perc*100:.2f} % of the records remain after outliers have been removed.")

98.12 % of the records remain after outliers have been removed.


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [34]:
taxi_jan_filtered.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [35]:
taxi_jan_filtered['DOLocationID'].info()

<class 'pandas.core.series.Series'>
Index: 3009173 entries, 0 to 3066765
Series name: DOLocationID
Non-Null Count    Dtype
--------------    -----
3009173 non-null  int64
dtypes: int64(1)
memory usage: 45.9 MB


In [36]:
# Turn the dataframe into a list of dictionaries
df_dict = taxi_jan_filtered[['PULocationID', 'DOLocationID']].to_dict('records')

In [37]:
# Fit a dictionary vectorizer
from sklearn.feature_extraction import DictVectorizer

vec = DictVectorizer(sparse=False)
vec.fit(df_dict)

DictVectorizer(sparse=False)

In [38]:
# Get a feature matrix from it
feature_matrix = vec.transform(df_dict)

print('The dimensionality of the matrix is:', feature_matrix.shape[1])

The dimensionality of the matrix is: 2


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [39]:
# Import linearRegression and rmse
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# obtain target data values (we already have the training data)
target = taxi_jan_filtered['duration'].values

# fit default linear regression model
model = LinearRegression()
model.fit(feature_matrix, target)

LinearRegression()

In [40]:
# obtain predictions on training data
y_pred = model.predict(feature_matrix)

In [41]:
# calculate the RMSE
rmse = np.sqrt(mean_squared_error(target, y_pred))
print(f"Root Mean Squared Error on train data: {rmse:.3f}")

Root Mean Squared Error on train data: 9.835


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022). 

What's the RMSE on validation?

In [43]:
# first, we need to transform the data in the same way as we have done for january data
# create duration variable computing the difference between dropoff and pickup times
taxi_feb['duration'] = taxi_feb.tpep_dropoff_datetime - taxi_feb.tpep_pickup_datetime 

# convert it to minutes
taxi_feb['duration'] = taxi_feb['duration'].dt.total_seconds()/60

# remove outliers
taxi_feb_filtered = taxi_feb[(taxi_feb['duration'] >= 1.0) & (taxi_feb['duration'] <= 60.0)]

# obtain target values for validation data
validation_target = taxi_feb_filtered['duration'].values

# one hot encoding
# Turn the dataframe into a list of dictionaries
df_dict_feb = taxi_feb_filtered[['PULocationID', 'DOLocationID']].to_dict('records')

# Fit a dictionary vectorizer
vec_feb = DictVectorizer(sparse=False)
vec_feb.fit(df_dict_feb)

# Get a feature matrix from it
feature_matrix_feb = vec.transform(df_dict_feb)

# obtain predicitons with validation data
y_pred_validation = model.predict(feature_matrix_feb)

# calculate the RMSE
rmse = np.sqrt(mean_squared_error(validation_target, y_pred_validation))
print(f"Root Mean Squared Error on validation data: {rmse:.3f}")

Root Mean Squared Error on validation data: 9.964
